In [2]:
#[derive(Clone, Copy)]
enum DiffNode<'a>{
    Add(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Sub(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Mul(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Div(&'a DiffNode<'a>, &'a DiffNode<'a>),
    Value(f32),
    WithSymbol(&'a DiffNode<'a>, &'static str),
}
use DiffNode::*;

impl<'a> DiffNode<'a>{
    fn has_symbol(&'a self, symbol: &'static str) -> bool{
        if let WithSymbol(_, sym) = self{
            sym==&symbol
        }else{false}
    }

    fn eval(&'a self) -> f32{
        match self{
            Add(a, b) => a.eval() + b.eval(),
            Sub(a, b) => a.eval() - b.eval(),
            Mul(a, b) => a.eval() * b.eval(),
            Div(a, b) => a.eval() / b.eval(),
            Value(v) => *v,
            WithSymbol(n, _) => n.eval()
        }
    }
    
    fn derivative(&'a self, x: &'static str) -> f32{
        match self{
            Add(a, b) => a.derivative(x) + b.derivative(x),
            Sub(a, b) => a.derivative(x) + -b.derivative(x),
            Mul(a, b) => a.derivative(x) * b.eval() + b.derivative(x) * a.eval(),
            
            // WRONG!
            Div(a, b) => a.derivative(x) / b.eval() - b.derivative(x) / a.eval(),
            
            Value(_) => 0.,
            WithSymbol(inner, sym) => if sym == &x{1.}else{inner.derivative(sym)}
        }
    }
    
    fn with_symbol(&'a self, sym: &'static str) -> Self{
        WithSymbol(self, sym)
    }
}

In [3]:
let x = Value(2.).with_symbol("x");

let y = Sub(&Add(&Value(1.), &Mul(&Add(&x, &Value(2.)), &Add(&Value(3.), &Value(3.)))), &x).derivative("x");

println!("{y}");

5


In [8]:
#![feature(type_name_of_val, inline_const)]
fn bruh(){}
std::any::type_name_of_val(&std::any::type_name_of_val::<u8>);

fn name(v: &dyn std::any::Any){
    println!("{}", std::any::type_name_of_val(v))
}

name(&"bruh");

fn is_closure<'a>(name: &'a str) -> bool{
    //const {for _ in 0..10{}};
    const CLOSURE: &str = "{{closure}}";
    &name[name.len()-CLOSURE.len()..] == CLOSURE
}

println!("{}", is_closure(std::any::type_name_of_val(&||true)));
println!("{}", std::any::type_name_of_val(&||false));
&"okok"[2..]

Error: `for` is not allowed in a `const`

In [5]:
use std::any::Any;
bruh.type_id()

TypeId { t: 17966091366821032593 }

In [6]:
trait SimpleVec<F>{
    fn as_ref(&self) -> &[F];

    // This should be moved to a `Dot` trait or smth
    fn dot(&self, other: impl SimpleVec<F>) -> F where F: std::ops::Mul<F, Output=F> + std::iter::Sum + Copy{
        self.as_ref().iter().zip(other.as_ref().iter()).map(|(a,b)| *a**b).sum()
    }
}

impl SimpleVec<f32> for [f32]{
    fn as_ref(&self) -> &[f32]{
        self
    }
}

impl<const LEN: usize> SimpleVec<f32> for [f32; LEN]{
    fn as_ref(&self) -> &[f32]{
        self
    }
}

[1f32, 2., 3.].dot([2f32, 3., 4.])

20.0